In [1]:
import pandas as pd
import numpy as np

print("Pandas Version: " + pd.__version__)
print("Numpy Version: " + np.__version__)

Pandas Version: 1.2.4
Numpy Version: 1.18.5


In [2]:
trips = pd.read_csv("../data/clean/trips.csv", parse_dates=["date_from", "date_until"])
trips["time_since_last_checkout"] = pd.to_timedelta(trips["time_since_last_checkout"])
trips.head(3)

,bike_id,user_id,date_from,date_until,start_station_name,start_station_id,end_station_name,end_station_id,booked_via,duration_in_min,distance_in_km,speed_in_kmh,time_since_last_checkout
0,119911,341973A96CDE0DF9792F6C844622735AE4216EBF,2014-01-01 00:02:51,2014-01-01 00:20:04,Enckeplatz / Hütten,131887,Königstraße / Struenseestraße,131650,Terminal HH_8 (-2624-),18,2.364129,7.880431,NaT
1,118994,665D79F269FA03F84FC61F3A7F7B078D7392EC0E,2014-01-01 00:07:45,2014-01-01 00:10:48,Isestraße / Hoheluftbrücke,140804,Isestraße / Hoheluftbrücke,140804,Terminal HH_63 (-2241-),4,0.000000,0.000000,NaT
2,143660,B46B52FDC494E46849DB84BF84F0B99C78358E59,2014-01-01 00:09:55,2014-01-01 00:26:20,Schulterblatt/Eifflerstraße,131648,Schulterblatt/Eifflerstraße,131648,Android SRH,17,0.000000,0.000000,NaT


In [71]:
user_groups = trips.groupby(["user_id"])

In [72]:
last_moment = trips.date_until.max()

In [73]:
user_trip_dates = trips.groupby(["user_id"], as_index=False).agg({"date_from": [np.min, np.max]})
user_trip_dates.columns = pd.Index(["user_id", "date_from_min", "date_from_max"])
user_trip_dates.set_index("user_id", inplace=True)

In [74]:
trips_count = user_groups.size().to_frame()
trips_count.rename(columns = {0: "trips_count"}, inplace=True)

In [75]:
active_period_in_days = user_groups.apply(
    lambda x: (x.date_from.max() - x.date_from.min()).days
).to_frame()
active_period_in_days.rename(columns = {0: "active_period_in_days"}, inplace=True)

In [76]:
days_since_first_activity = user_groups.apply(
    lambda x: (last_moment - x.date_from.min()).days
).to_frame()
days_since_first_activity.rename(columns = {0: "days_since_first_activity"}, inplace=True)

In [77]:
days_since_last_activity = user_groups.apply(
    lambda x: (last_moment - x.date_from.max()).days
).to_frame()
days_since_last_activity.rename(columns = {0: "days_since_last_activity"}, inplace=True)

In [78]:
# exclude all <= 0 (round trips)
mean_duration_in_min = trips[["user_id", "duration_in_min"]][trips.duration_in_min > 0].groupby("user_id").mean()
mean_duration_in_min.rename(columns = {"duration_in_min": "mean_duration_in_min"}, inplace=True)

In [79]:
# exclude all <= 0 (round trips)
mean_distance_in_km = trips[["user_id", "distance_in_km"]][trips.distance_in_km > 0].groupby("user_id").mean()
mean_distance_in_km.rename(columns = {"distance_in_km": "mean_distance_in_km"}, inplace=True)

In [80]:
# exclude all <= 0 (round trips)
mean_speed_in_kmh = trips[["user_id", "speed_in_kmh"]][trips.speed_in_kmh > 0].groupby("user_id").mean()
mean_speed_in_kmh.rename(columns = {"speed_in_kmh": "mean_speed_in_kmh"}, inplace=True)

In [81]:
users = pd.merge(user_trip_dates, trips_count, left_index=True, right_index=True)
users = pd.merge(users, active_period_in_days, left_index=True, right_index=True)
users = pd.merge(users, days_since_first_activity, left_index=True, right_index=True)
users = pd.merge(users, days_since_last_activity, left_index=True, right_index=True)
users = pd.merge(users, mean_duration_in_min, left_index=True, right_index=True)
users = pd.merge(users, mean_distance_in_km, left_index=True, right_index=True)
users = pd.merge(users, mean_speed_in_kmh, left_index=True, right_index=True)

In [83]:
# Calculate the mean time between trips in hours

mean_time_between_trips = trips[["user_id", "time_since_last_checkout"]].dropna()
mean_time_between_trips = mean_time_between_trips.groupby("user_id").mean(numeric_only=False)

users = pd.merge(
    users, mean_time_between_trips, left_index=True, right_index=True
).rename(columns={"time_since_last_checkout": "mean_time_between_trips"})

In [84]:
users.sample(5)

,date_from_min,date_from_max,trips_count,active_period_in_days,days_since_first_activity,days_since_last_activity,mean_duration_in_min,mean_distance_in_km,mean_speed_in_kmh,trips_per_day,mean_time_between_trips
user_id,,,,,,,,,,,
FB78BD8C8B31B99107E0A1E6C555C9C92079A1D7,2014-12-02 13:24:32,2015-04-19 05:06:05,8,137,895,758,20.750000,1.638491,4.879532,0.058394,19 days 15:57:21.857142857
6DE5BB050608A3267AFA9A74794025788BA0ACBD,2016-06-30 12:18:19,2016-06-30 13:16:41,2,0,319,319,22.000000,3.521846,9.605034,inf,0 days 00:58:22
16FD1518370798D2041CD421B9EACAA98C64CBED,2016-06-22 19:10:25,2016-09-12 16:05:50,2,81,327,245,24.500000,3.450320,8.305916,0.024691,81 days 20:55:25
9D9A9BD750BE9E743970A58C84B7D9E8738661A7,2015-06-11 17:17:49,2017-04-18 08:17:49,145,676,704,27,13.827586,1.764972,8.564981,0.214497,4 days 16:46:15
62E79C0D252A9841A34527D06631AD14C0C017D2,2014-06-27 13:04:52,2014-08-06 20:28:41,4,40,1053,1013,10.000000,1.469707,8.573631,0.100000,13 days 10:27:56.333333333


In [85]:
users.to_csv("../data/clean/users.csv", index=True)